# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [2]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Create a model
The LUME-serives Model object handles all model operations.

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)
model

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2022, 9, 17, 2, 51, 58),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu_hxr',                     description='test_model'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [5]:
# create a project
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)
scheduling_service.create_project("test")

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


You can now find this project in you Prefect UI at http://localhost:8080


![project](https://slaclab.github.io/lume-services/files/project_nav.png)

## Create a deployment for your model

In [6]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.38/my_model-0.0.38.tar.gz"

# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.models.model:installing package
INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.38/my_model-0.0.38.tar.gz saved to /tmp/lume-services/sources/my_model-0.0.38.tar.gz
INFO:lume_services.environment.solver:Uninstall complete
INFO:lume_services.environment.solver:Installing dependencies...


Found existing installation: my-model 0.0.37
Uninstalling my-model-0.0.37:
  Successfully uninstalled my-model-0.0.37



INFO:lume_services.environment.solver:Dependency installation complete
  Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-zhlew2pq/my_model.egg-info
  writing /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-zhlew2pq/my_model.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-zhlew2pq/my_model.egg-info/dependency_links.txt
  writing entry points to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-zhlew2pq/my_model.egg-info/entry_points.txt
  writing requirements to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-zhlew2pq/my_model.egg-info/requires.txt
  writing top-level names to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-zhlew2pq/my_model.egg-info/top_level.txt
  writing manifest file '/private/var/folders/nh/g2v_nm

Flow URL: http://localhost:8080/default/flow/82872f15-f0cf-48fb-82f7-08eee2631795
 └── ID: 34e60fcb-1e08-455b-affd-7236e6c5efbd
 └── Project: test
 └── Labels: ['lume-services']


## Run the Prefect workflow directly

In [ ]:
model.deployment.flow.prefect_flow.run(**{
                        "input1": 1, 
                        "input2": 2, 
                        "filename": "/Users/jacquelinegarrahan/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"
    }
)

## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory. 
After running the next cell, you'll be able to see the running container in your docker desktop and examine the flow using the Prefect UI at http://localhost:8080/default?flows.

In [8]:
model.run(
    parameters={
        "input1": 1, 
        "input2": 2, 
        "filename": "/lume-services/data/test_file.txt", 
        "filesystem_identifier":"mounted"}
)

In [ ]:
# TODO: RESULTS INTERFACE

## Load model using model id

In [ ]:
from lume_services.models import Model

loaded_model = Model(model_id=1)

In [ ]:
model.deployment

In [ ]:
loaded_model.load_deployment()

In [ ]:
loaded_model.deployment

In [ ]:
loaded_model.run_and_return(parameters={
                        "input1": "hey", 
                        "input2": "jackie", 
                        "filename": f"/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"
        }
)

In [ ]:
model.get_results()

In [ ]:
import my_model
my_model.__version__